In [5]:

import tensorflow as tf
import numpy as np 

import os
import sys

# append src to python path, access it as a package
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data import load_fer




In [6]:
labeled_data = load_fer.load_fer()


NameError: in user code:

    /local_drive/src/data/load_fer.py:29 process_path  *
        img = decode_img(img)
    /local_drive/src/data/load_fer.py:20 decode_img  *
        return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

    NameError: name 'IMG_HEIGHT' is not defined


In [ ]:
for f in list_ds.take(5):
  print(f.numpy())


In [ ]:


for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())


In [ ]:

# single example 
single_image = list(list_ds.take(5))[0] 
